In [23]:
!pip install megadetector torch torchvision pillow fiftyone pybioclip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 31.2 MB/s eta 0:00:00


In [27]:
import os
from google.colab import drive
import fiftyone as fo
from megadetector.utils import url_utils
from megadetector.visualization import visualization_utils as vis_utils
from megadetector.detection import run_detector
import pandas as pd
import numpy as np
import torch
from torchvision.ops import nms
from bioclip import CustomLabelsClassifier, CustomLabelsBinningClassifier, TreeOfLifeClassifier, Rank


In [4]:
# Mount your data that is in google drive
drive.mount('/content/drive')
# os.listdir('/content/drive/MyDrive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['Colab Notebooks',
 'Forensics.gdoc',
 'Attempt 2.wlmp',
 'Christianity.gdoc',
 'Philosphy Paper.docx',
 'Film as Literature Final.gdoc',
 'The erotes are a group of winged gods in Classical mythology.docx',
 'My Saved Places.gmap',
 'I-9 and instructions - 2015.pdf',
 'Cambodia Essay.gdoc',
 'Untitled document (7).gdoc',
 'Untitled document (6).gdoc',
 'Mini Trail Guide Brainstorm.gdoc',
 'Camping gear and locations.gdoc',
 'NZ.xlsx',
 'Important Documents',
 'Whanganui.gsheet',
 'Break plan.gsheet',
 'NZ.gsheet',
 'Potential Research Locations.gmap',
 '20190827161612851.pdf',
 'collections.pdf',
 'Summer break plans.gsheet',
 'Summer break budget.gsheet',
 'BasicThesis.nlogo',
 'IMG_3345.JPG',
 'IMG_3341.JPG',
 'IMG_3344.JPG',
 'IMG_3338.JPG',
 'IMG_3350.JPG',
 'IMG_3337.JPG',
 'IMG_3348.JPG',
 'IMG_3346.JPG',
 'IMG_3347.JPG',
 'IMG_3335.JPG',
 'IMG_3340.JPG',
 'IMG_3343.JPG',
 'IMG_3342.JPG',
 'IMG_3339.JPG',
 'IMG_2977.JPG',
 'IMG_3353.JPG',
 'IMG_3336.JPG',
 'IMG_3334.JPG',
 'IMG

In [5]:
# Create a fiftyone dataset
name = 'myDataset'
dir = '/content/drive/MyDrive/example_images/'
if fo.dataset_exists(name):
  dataset = fo.load_dataset(name)
else:
  dataset = fo.Dataset.from_images_dir(dir)
  dataset.name = name
  dataset.compute_metadata()
  dataset.save()


 100% |███████████████████| 10/10 [28.0ms elapsed, 0s remaining, 357.8 samples/s]     


INFO:eta.core.utils: 100% |███████████████████| 10/10 [28.0ms elapsed, 0s remaining, 357.8 samples/s]     


Computing metadata...


INFO:fiftyone.core.metadata:Computing metadata...


 100% |███████████████████| 10/10 [1.9s elapsed, 0s remaining, 5.3 samples/s]      


INFO:eta.core.utils: 100% |███████████████████| 10/10 [1.9s elapsed, 0s remaining, 5.3 samples/s]      


In [26]:
# Prepare detection model
md_model = run_detector.load_detector('MDV5A')  # Automatically downloads model weights
md_model_name = "MegaDet"
md_class_map = {
    "1": "animal",
    "2": "human",
    "3": "vehicle"
}
# Enter option 3

Model v5a.0.1 already exists and is valid at /tmp/megadetector_models/md_v5a.0.1.pt
Bypassing imports for model type yolov5
Loading PT detector with compatibility mode classic
Loaded image size 1280 from model metadata
Using model stride: 64


INFO:yolov5:Fusing layers... 
Fusing layers... 
Fusing layers... 
INFO:yolov5:Model summary: 733 layers, 140054656 parameters, 0 gradients, 208.8 GFLOPs
Model summary: 733 layers, 140054656 parameters, 0 gradients, 208.8 GFLOPs
Model summary: 733 layers, 140054656 parameters, 0 gradients, 208.8 GFLOPs


In [29]:
# Prepare classification model
#classes format... "models_prompt": "class_name"
# model_device = "cuda:0" # If you are on a GPU runtime
model_device = "cpu" # If you are on a non-GPU runtime, MUCH SLOWER
classes = {
	"elk": "ungulate",
	"moose": "ungulate",
	"dog": "canine",
	"wolf": "canine",
  "human": "human"
}
bc_model = CustomLabelsBinningClassifier(classes, device=model_device)
# bc_model = TreeOfLifeClassifier(device=model_device)
bc_model_name = "bioclip"

INFO:root:Parsing model identifier. Schema: hf-hub, Identifier: imageomics/bioclip-2
Parsing model identifier. Schema: hf-hub, Identifier: imageomics/bioclip-2
Parsing model identifier. Schema: hf-hub, Identifier: imageomics/bioclip-2
INFO:root:Attempting to load config from HF Hub: imageomics/bioclip-2
Attempting to load config from HF Hub: imageomics/bioclip-2
Attempting to load config from HF Hub: imageomics/bioclip-2
INFO:root:Loaded model config from HF Hub: imageomics/bioclip-2
Loaded model config from HF Hub: imageomics/bioclip-2
Loaded model config from HF Hub: imageomics/bioclip-2
INFO:root:Found default weights file on HF Hub: /root/.cache/huggingface/hub/models--imageomics--bioclip-2/snapshots/df00ae6ffea92819f318258352712c8cc3d1d5b3/open_clip_model.safetensors
Found default weights file on HF Hub: /root/.cache/huggingface/hub/models--imageomics--bioclip-2/snapshots/df00ae6ffea92819f318258352712c8cc3d1d5b3/open_clip_model.safetensors
Found default weights file on HF Hub: /ro

In [34]:
results = []
for sample in dataset.iter_samples(progress = True, autosave = True):
  # Load the sample
  img = vis_utils.load_image(sample.filepath)
  img_w = sample.metadata['width']
  img_h = sample.metadata['height']
  # Run MegaDetector
  md_pred = md_model.generate_detections_one_image(img, detection_threshold=0.1)
  # Convert detections to the format expected by NMS
  boxes = [d['bbox'] for d in md_pred['detections']]
  scores = [d['conf'] for d in md_pred['detections']]
  boxes_tensor = torch.tensor(boxes)
  scores_tensor = torch.tensor(scores)
  # Convert boxes to [x1, y1, x2, y2] format
  boxes_xyxy = torch.tensor([[x, y, x + w, y + h] for x, y, w, h in boxes])
  keep = nms(boxes_xyxy, scores_tensor, iou_threshold=0.3)
  filtered_detections = [md_pred['detections'][i] for i in keep]
  md_detections = []
  bc_classifications = []
  for det in filtered_detections:
      md_detections.append(
          fo.Detection(
            label=md_class_map[det['category']],
            bounding_box=det['bbox'],
            confidence=det["conf"],
        )
      )
      # Get the cropped detection
      x, y, w, h = det['bbox']
      xmin, xmax = int(np.floor(img_w * x)), int(np.ceil(img_w * (x + w)))
      ymin, ymax = int(np.floor(img_h * y)), int(np.ceil(img_h * (y + h)))
      det_crop = img.crop((xmin, ymin, xmax, ymax))
      # Classify the cropped detection with BioCLIP
      bc_preds = bc_model.predict([det_crop], k=1)
      # IF USING THE FULL TREE OF LIFE
      # bc_preds = bc_model.predict([det_crop], rank=Rank.SPECIES, k = 1)
      # Save the classification as a detection overlapping with
      # the MegaDetector detection
      bc_classifications.append(
        fo.Detection(
          label=bc_preds[0]['classification'],
          bounding_box=det['bbox'],
          confidence=bc_preds[0]["score"],
        )
      )
  sample[md_model_name] = fo.Detections(detections=md_detections)
  sample[bc_model_name] = fo.Detections(detections=bc_classifications)


   0% ||------------------|  0/10 [13.3ms elapsed, ? remaining, ? samples/s] 

100%|██████████| 1/1 [00:03<00:00,  3.57s/images]

  10% |█/-----------------|  1/10 [31.9s elapsed, 4.8m remaining, 0.0 samples/s] 


100%|██████████| 1/1 [00:04<00:00,  4.39s/images]

  20% |███----------------|  2/10 [1.1m elapsed, 4.2m remaining, 0.0 samples/s]  

...


100%|██████████| 1/1 [00:03<00:00,  3.15s/images]

  90% |█████████████████/-|  9/10 [4.7m elapsed, 31.5s remaining, 0.0 samples/s] 


100%|██████████| 1/1 [00:03<00:00,  3.13s/images]

 100% |███████████████████| 10/10 [5.1m elapsed, 0s remaining, 0.0 samples/s]    



INFO:eta.core.utils: 100% |███████████████████| 10/10 [5.1m elapsed, 0s remaining, 0.0 samples/s]    
 100% |███████████████████| 10/10 [5.1m elapsed, 0s remaining, 0.0 samples/s]    
 100% |███████████████████| 10/10 [5.1m elapsed, 0s remaining, 0.0 samples/s]    


In [36]:
session = fo.launch_app(dataset, remote=False, port=1122)

In [1]:
results = []
for sample in dataset.iter_samples(progress = True, autosave = True):
  for field in [md_model_name, bc_model_name]:
    if sample[field] is not None:
      for det in sample[field].detections:
        results.append([
          sample.filepath,
          field,
          det.label,
          det.confidence,
          det.bounding_box[0],
          det.bounding_box[1],
          det.bounding_box[2],
          det.bounding_box[3]
        ])
dataframe = pd.DataFrame(
    results,
    columns=[
        'filepath',
        'model',
        'category',
        'confidence',
        'xmin',
        'ymin',
        'xmax',
        'ymax'
    ]
)
print(dataframe)
dataframe.to_csv("demo_output/batch_output.csv", index = False)

NameError: name 'url_utils' is not defined